In [1]:
from src.data_synthesis import QuestionTemplate, SQLConditionTemplate, SQLColumnExpression, SQLOperatorTemplate, SQLTemplate

In [17]:
from unittest.mock import Mock
from src.data_synthesis import MIN, MAX, AVG, SUM, NOOP
nl = "What is the {operator} of column {col1} given that {col2} has value {value}?"
main = SQLColumnExpression(("{col1}",))
conditions = [SQLConditionTemplate('{col2}', '=', '{value}')]
allowed_operators = tuple([MIN, MAX, AVG, SUM, NOOP])
schema={'col1':{'type':'collumn',
                'allowed_dtypes':['numeric']
                },
        'col2':{'type':'collumn',
                'allowed_dtypes':['numeric']
               },
        'val1':{'type':'value',
                'allowed_dtypes':['numeric']
               },
        'sample_strategy':'random',
        'value_pool':'distinct_values',
        'interpolation_args':dict()
       }
dummy_table = Mock()
dummy_table.column_values = lambda x, **kwargs: list(range(len(x)))
dummy_table.column_names = ['num_col1', 'num_col2', 'txt_col1', 'txt_col2']
dummy_table.infer_column_type = lambda x: 'numeric'
temp = QuestionTemplate(nl, main, conditions, allowed_operators, schema)

In [3]:
conditions[0].generate().format(col2 = 'score', value = "'1-0'")

"\n\tAND score = '1-0'"

In [7]:
compiled_sql = SQLTemplate(SQLOperatorTemplate('max', main), conditions).format(col1 = 'team_id', col2 = 'score', value = "'1-0'").generate()
print(compiled_sql)

SELECT max(team_id)FROM df
WHERE true
	AND score = '1-0'


In [29]:
import itertools
col_names = ['a', 'b', 'c']
assignments = list(itertools.product(*[col_names for i in range(3)]))
print(assignments)
print()
assignments2= list(itertools.permutations(col_names, 3))
print(assignments2)

[('a', 'a', 'a'), ('a', 'a', 'b'), ('a', 'a', 'c'), ('a', 'b', 'a'), ('a', 'b', 'b'), ('a', 'b', 'c'), ('a', 'c', 'a'), ('a', 'c', 'b'), ('a', 'c', 'c'), ('b', 'a', 'a'), ('b', 'a', 'b'), ('b', 'a', 'c'), ('b', 'b', 'a'), ('b', 'b', 'b'), ('b', 'b', 'c'), ('b', 'c', 'a'), ('b', 'c', 'b'), ('b', 'c', 'c'), ('c', 'a', 'a'), ('c', 'a', 'b'), ('c', 'a', 'c'), ('c', 'b', 'a'), ('c', 'b', 'b'), ('c', 'b', 'c'), ('c', 'c', 'a'), ('c', 'c', 'b'), ('c', 'c', 'c')]

[('a', 'b', 'c'), ('a', 'c', 'b'), ('b', 'a', 'c'), ('b', 'c', 'a'), ('c', 'a', 'b'), ('c', 'b', 'a')]


In [4]:
import timeit
print(timeit.timeit("for i in l: pass", "l = [1,4,7,10,10,10,10,10,13]"))
print(timeit.timeit("for i in s: pass", "s = set([1,4,7,10,10,10,10,10,13])"))
print(timeit.timeit("for i in ls: pass", "ls = list(set([1,4,7,10,10,10,10,10,13]))"))

0.07656779998796992
0.07621179999841843
0.04616130000795238


In [12]:
a = int(float('inf'))
b = 5
print(a>b)

OverflowError: cannot convert float infinity to integer

In [16]:
import numpy as np
list_a = [1,2,3,4,5,6]
indices = np.array([5,3,4,1,2], dtype=int)
array_a = np.array(list_a)
print(list(array_a[indices]))

[6, 4, 5, 2, 3]


In [7]:
list_b = [(1,2,3),(4,5)]
print([[elem for elem in tup] for tup in list_b])

[[1, 2, 3], [4, 5]]


In [3]:
from src.data_synthesis import find_template_variables
sql_tmplte = 'select {col1} from df where true and {col2} = {val1}'
var_list = find_template_variables(sql_tmplte)
var_list[0].strip('{}')

'col1'

In [4]:
variables = var_list
column_variables = [variable for variable in variables
                            if schema[variable]['type'] == 'collumn']
column_variables

['col1', 'col2']

In [5]:
import itertools
column_assignments = list(itertools.permutations(dummy_table.column_names,len(column_variables)))
column_assignments

[('num_col1', 'num_col2'),
 ('num_col1', 'txt_col1'),
 ('num_col1', 'txt_col2'),
 ('num_col2', 'num_col1'),
 ('num_col2', 'txt_col1'),
 ('num_col2', 'txt_col2'),
 ('txt_col1', 'num_col1'),
 ('txt_col1', 'num_col2'),
 ('txt_col1', 'txt_col2'),
 ('txt_col2', 'num_col1'),
 ('txt_col2', 'num_col2'),
 ('txt_col2', 'txt_col1')]

In [6]:
import time
test_num = 100000
time1 = 0
for i in range(test_num):
    start1 = time.time()
    column_assignments = list(set([assignment 
                                   for assignment in column_assignments
                                   for c, col in enumerate(assignment)
                                   if dummy_table.infer_column_type(col) in schema[column_variables[c]]['allowed_dtypes']
                                  ]
                                 )
                             )
    end1 = time.time()
    time1 += (end1-start1)/test_num
print(time1)

time2 = 0
for i in range(test_num):
    start2 = time.time()
    result = []
    for assignment in column_assignments:
        compliant = 0
        for c, col in enumerate(assignment):
            if dummy_table.infer_column_type(col) in schema[column_variables[c]]['allowed_dtypes']:
                compliant += 1
        if compliant == c+1:
            result.append(assignment)
    end2=time.time()
    time2 += (end2-start2)/test_num
print(time2)
sorted(column_assignments) == sorted(result)

1.338667392731476e-05
2.3070974349984597e-05


True

In [7]:
conditions = [Mock(), Mock()]
conditions[0].condition_col = 'col1'
conditions[1].condition_col = 'col2'
condition_vars = [condition.condition_col for condition in conditions]
condition_ids = [idx for idx, col_var in enumerate(column_variables) if col_var in condition_vars]
print(condition_vars)
print(condition_ids)

['col1', 'col2']
[0, 1]


In [8]:
# b) set of all column names at that ids after dtype check
condition_assignments = list(set([assignment[idx] for assignment in column_assignments for idx in condition_ids]))
# all combinations of condition col var assignments
# condition_assignments = list(set([tuple([assignment[idx] for idx in condition_ids]) for assignment in column_assignments]))
condition_assignments

['num_col2', 'num_col1', 'txt_col1', 'txt_col2']

In [9]:
col_values = [dummy_table.column_values(col, distinct=True) for col in list(set(condition_assignments))]
num_col_values = [len(col) for col in col_values]
print(col_values)
print(num_col_values)

[[0, 1, 2, 3, 4, 5, 6, 7], [0, 1, 2, 3, 4, 5, 6, 7], [0, 1, 2, 3, 4, 5, 6, 7], [0, 1, 2, 3, 4, 5, 6, 7]]
[8, 8, 8, 8]


In [10]:
import numpy as np
sample_idxs = np.random.choice(range(min(num_col_values)),
                                        min(min(num_col_values), 10),
                                        replace=False
                                        )
sample_idxs

array([0, 4, 6, 7, 2, 1, 3, 5])

In [11]:
strategy = 'random'
max_samples = 10
num_draws = max_samples
shuffle = True
sample_idxs = np.zeros(max_samples, dtype=int)
filled_samples = 0
# if mix strategy fill only remaining half of sample budget with random samples
sample_num = max_samples if strategy == 'random' else max_samples // 2
for draw in range(num_draws):
    if filled_samples >= 10:
        break
    chosen_idxs = np.random.choice(range(min(num_col_values)),
                                min(min(num_col_values), max_samples),
                                replace=False
                                )
    if not shuffle and draw == 0:  # TODO maybe delete second condition draw==0 since joint sampling achieved by colum_names as lists
        sample_idxs.sort()

    if filled_samples + len(chosen_idxs) > max_samples:
        chosen_idxs = chosen_idxs[:max_samples-filled_samples-1]

    sample_idxs[filled_samples:filled_samples+len(chosen_idxs)] = chosen_idxs
    filled_samples += len(chosen_idxs)
sample_idxs

array([0, 1, 7, 2, 4, 5, 6, 3, 4, 0])

In [12]:
from src.data_synthesis import sample_values
num_value_samples = 10
# d) iterate over results from b and call sample save in dict
samples = {condition_col:sample_values(dummy_table,
condition_col,
max_samples=num_value_samples,
num_draws=num_value_samples,
strategy=schema['sample_strategy'],
value_pool=schema['value_pool'],
shuffle=True,
**schema['interpolation_args'],
return_indices=False
) 
for condition_col in condition_assignments
}

In [13]:
samples

{'num_col2': [7, 2, 6, 5, 1, 0, 3, 4, 1, 0],
 'num_col1': [4, 0, 3, 1, 5, 7, 2, 6, 5, 0],
 'txt_col1': [2, 7, 1, 5, 4, 6, 0, 3, 3, 0],
 'txt_col2': [4, 1, 6, 7, 3, 2, 5, 0, 7, 0]}

In [14]:
tmp1=[[assignment[i] 
for i in condition_ids
]
for assignment in column_assignments]
print(tmp1)
tmp2=[samples[condition_col] 
for condition_col in [assignment[i] 
for i in condition_ids
]
]
print(tmp2)
value_assignments = [zip(*[samples[condition_col] 
for condition_col in [assignment[i] 
for i in condition_ids
]
]
)
for assignment in column_assignments
]
value_assignments

[['txt_col2', 'num_col1'], ['txt_col2', 'txt_col1'], ['txt_col1', 'txt_col2'], ['num_col1', 'txt_col2'], ['num_col2', 'num_col1'], ['num_col2', 'txt_col1'], ['txt_col1', 'num_col2'], ['txt_col2', 'num_col2'], ['num_col1', 'num_col2'], ['txt_col1', 'num_col1'], ['num_col2', 'txt_col2'], ['num_col1', 'txt_col1']]
[[4, 0, 3, 1, 5, 7, 2, 6, 5, 0], [2, 7, 1, 5, 4, 6, 0, 3, 3, 0]]


In [15]:
[value for value in value_assignments[0]]

[(4, 4),
 (1, 0),
 (6, 3),
 (7, 1),
 (3, 5),
 (2, 7),
 (5, 2),
 (0, 6),
 (7, 5),
 (0, 0)]

In [16]:
all_assignments = [tuple([*assignment, *value])
                        for i, assignment in enumerate(column_assignments)
                        for value in value_assignments[i]
                        ]
all_assignments

[('txt_col2', 'txt_col1', 4, 2),
 ('txt_col2', 'txt_col1', 1, 7),
 ('txt_col2', 'txt_col1', 6, 1),
 ('txt_col2', 'txt_col1', 7, 5),
 ('txt_col2', 'txt_col1', 3, 4),
 ('txt_col2', 'txt_col1', 2, 6),
 ('txt_col2', 'txt_col1', 5, 0),
 ('txt_col2', 'txt_col1', 0, 3),
 ('txt_col2', 'txt_col1', 7, 3),
 ('txt_col2', 'txt_col1', 0, 0),
 ('txt_col1', 'txt_col2', 2, 4),
 ('txt_col1', 'txt_col2', 7, 1),
 ('txt_col1', 'txt_col2', 1, 6),
 ('txt_col1', 'txt_col2', 5, 7),
 ('txt_col1', 'txt_col2', 4, 3),
 ('txt_col1', 'txt_col2', 6, 2),
 ('txt_col1', 'txt_col2', 0, 5),
 ('txt_col1', 'txt_col2', 3, 0),
 ('txt_col1', 'txt_col2', 3, 7),
 ('txt_col1', 'txt_col2', 0, 0),
 ('num_col1', 'txt_col2', 4, 4),
 ('num_col1', 'txt_col2', 0, 1),
 ('num_col1', 'txt_col2', 3, 6),
 ('num_col1', 'txt_col2', 1, 7),
 ('num_col1', 'txt_col2', 5, 3),
 ('num_col1', 'txt_col2', 7, 2),
 ('num_col1', 'txt_col2', 2, 5),
 ('num_col1', 'txt_col2', 6, 0),
 ('num_col1', 'txt_col2', 5, 7),
 ('num_col1', 'txt_col2', 0, 0),
 ('num_col

In [14]:
temp.find_all_possible_assignments(sql_tmplte, dummy_table)

[]

In [1]:
from src.data_synthesis import main
for tab in main():
    print(tab._table_id)
    print(tab.table_name)
    print(tab.size)
    print(f"{tab._source}-{tab._source_split}")
    df = tab.pandas_dataframe
    print(df.head(5))
    print(tab._inferred_column_types)
    print(tab.column_value_densitiy(tab.column_names[0]))

3ffac2f2b0a9e09025184d00017c8d85266d7c3814db35cd7d6a7bf33c2bf7e0
csv/203-csv/733.tsv
(5, 10)
wikitablequestions-test
  Rank                   Cyclist                Team        Time  \
0    1  Alejandro Valverde (ESP)    Caisse d'Epargne  5h 29' 10"   
1    2   Alexandr Kolobnev (RUS)  Team CSC Saxo Bank        s.t.   
2    3     Davide Rebellin (ITA)        Gerolsteiner        s.t.   
3    4       Paolo Bettini (ITA)          Quick Step        s.t.   
4    5   Franco Pellizotti (ITA)            Liquigas        s.t.   

  UCI ProTour\nPoints  
0                  40  
1                  30  
2                  25  
3                  20  
4                  15  
['numeric', 'text', 'text', 'text', 'numeric']
1.0
ca42a436db802301468c5d09a03749429faa827cb753dbfa09e254946fa8ca47
csv/204-csv/149.tsv
(8, 7)
wikitablequestions-test
                  Description Losses  1939/40  1940/41  1941/42  1942/43  \
0                  Direct War Losses  360,000                              
1          

In [4]:
from src.data_synthesis import main
results = main()
len(results)

415

In [2]:
from unittest.mock import Mock
mock1 = Mock()
mock1.name = 'm1'
mock2 = Mock()
mock2.name = 'm2'
mock_list = [mock1, mock2]

{(value := m).name:(m_name := value.name) for m in mock_list}

{'m1': 'm1', 'm2': 'm2'}

In [3]:
from pathlib import Path
cache_path_str = '../data/NumTabQA/.cache'
cache_path = Path(cache_path_str)
[path for path in cache_path.iterdir()]

[PosixPath('../data/NumTabQA/.cache/231017_1705_02_981459_table_list.pickle'),
 PosixPath('../data/NumTabQA/.cache/231017_1713_49_387622_wikitablequestions_train_tables.pickle'),
 PosixPath('../data/NumTabQA/.cache/231018_0937_07_542482_wikitablequestions_test_tables.pickle')]

In [1]:
import itertools
from src.data_synthesis import QuestionTemplate, SQLConditionTemplate, SQLColumnExpression, SQLOperatorTemplate, sql_template_from_components, main, find_template_variables
from src.data_synthesis import MIN, MAX, AVG, SUM, NOOP
results = main()
nl = "What is the {op} of column {col1} given that {col2} has value {val1}?"
main_expr = SQLColumnExpression(("{col1}",))
conditions = (SQLConditionTemplate(SQLColumnExpression(('{col2}',)), '=', '{val1}'),)
allowed_operators = tuple([MIN, MAX, AVG, SUM, NOOP])
schema = {'variables': {'col1': {'type':'column',
                                 'allowed_dtypes':['numeric']
                                 },
                        'col2':{'type':'column',
                                'allowed_dtypes':['numeric', 'text']
                               },
                        'val1':{'type':'value',
                                'allowed_dtypes':['numeric', 'text']
                               }
                        },
        'sample_strategy':'random',
        'value_pool':'distinct_values',
        'interpolation_args':dict()
       }
basic_template = QuestionTemplate(nl, main_expr, allowed_operators, conditions, schema)
sql_template_obj = SQLTemplate(
                    SQLOperatorTemplate(AVG.sql,
                                        main_expr,
                                        brackets=None,
                                        over_clause=None
                                        ),
                    conditions=conditions
                )
sql_template = sql_template_obj.generate()
print(sql_template)
table = list(results.values())[0]
variables = find_template_variables(sql_template)
print(variables)
column_variables = [variable for variable in variables
                            if schema['variables'][variable]['type'] == 'column']
# all permutations of column assignments
column_assignments = list(itertools.permutations(table.column_names,
                                                         len(column_variables)
                                                         ))
type_errors = set(
     [assignment
         for assignment in column_assignments
         for c, col in enumerate(assignment)
         if table.infer_column_type(col) not in 
          schema['variables'][column_variables[c]]['allowed_dtypes']
      ]
)

print(column_assignments := list(set(column_assignments) - type_errors))

#basic_template.find_all_possible_assignments(sql_template, table)
questions = basic_template.create_questions([list(results.values())[0]])
questions

SELECT avg("{col1}") FROM df
WHERE true
	AND "{col2}" = {val1}
['col1', 'col2', 'val1']
[('Rank', 'Cyclist'), ('UCI ProTour\\nPoints', 'Team'), ('Rank', 'Team'), ('UCI ProTour\\nPoints', 'Cyclist'), ('UCI ProTour\\nPoints', 'Time'), ('Rank', 'Time'), ('Rank', 'UCI ProTour\\nPoints'), ('UCI ProTour\\nPoints', 'Rank')]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10.29it/s]


In [2]:
for q in questions[:5]:
    q.compute_answer()
    print(q._nl_question)
    print(q.alternative_phrasings)
    print(q._sql_query)
    print(q._answer)
    print(q._num_conditions)
    print()
# TODO fill "" characters col escaping
    

What is the minimum of column Rank given that Team has value 
                                         'Euskaltel-Euskadi'
                                         ?
[]
SELECT min("Rank") FROM df
WHERE true
	AND "Team" = 
                                         'Euskaltel-Euskadi'
                                         
7
None

What is the minimum of column Rank given that Team has value 
                                         'Caisse d''Epargne'
                                         ?
[]
SELECT min("Rank") FROM df
WHERE true
	AND "Team" = 
                                         'Caisse d''Epargne'
                                         
1
None

What is the minimum of column Rank given that Team has value 
                                         'Ag2r-La Mondiale'
                                         ?
[]
SELECT min("Rank") FROM df
WHERE true
	AND "Team" = 
                                         'Ag2r-La Mondiale'
                                         
8
None

Wha

In [8]:
from contexttimer import Timer
big_list = [list(range(i)) for i in list(range(1000))]

with Timer() as t1:
    for a in range(1000):
        new_list = []
        for i in big_list:
            new_list += i
    print(t1.elapsed)
    
with Timer() as t2:
    for a in range(1000):
        new_list2 = [j for i in big_list for j in i]
    print(t2.elapsed)
    
new_list == new_list2

5.3149887000909075
9.386924999998882


True

In [13]:
list_c = [1,2,3,4,5] 
list_c += [1, 2,3]
list_c

[1, 2, 3, 4, 5, 1, 2, 3]

In [9]:

used_datatypes = [schema['variables'][variable]['allowed_dtypes']
                 for variable in schema['variables'].keys()
                 if schema['variables'][variable]['type'] == 'value'
                 ]
used_datatypes = set([dtype for elem in used_datatypes for dtype in elem])
            
used_datatypes

{'numeric', 'text'}

In [2]:
tup_a = (1,2,3,)
tup_b = ('col1', 'col2', 'col3',)
tuple(zip(tup_b, tup_a))

(('col1', 1), ('col2', 2), ('col3', 3))

In [10]:
from contexttimer import Timer
with Timer() as t1:
    for i in range(10000):
        res_a = tuple(zip(tup_a, tup_b))
    print(t1.elapsed)
    
with Timer() as t2:
    for i in range(10000):
        res_b = tuple([value for value in zip(tup_a, tup_b)])
    print(t2.elapsed)

res_a == res_b

0.007136399974115193
0.007497200043871999


True

In [5]:
print(zip_a := tuple(zip([1,2,3,4,5])))
print(zip_b := tuple(zip([1,2],[3,4])))

all_assignments = [assignment + value
                           for assignment in zip_b
                           for value in zip_a
                           ]

print(all_assignments)
zip_b = {var_name:zip_b[v] for v, var_name in enumerate(('a','b'))}

assignments = [{variable: assingnment[v]
                        for v, variable in enumerate(list(zip_b.keys()) + ['c'])
                        } for assingnment in all_assignments
                       ]

assignments

((1,), (2,), (3,), (4,), (5,))
((1, 3), (2, 4))
[(1, 3, 1), (1, 3, 2), (1, 3, 3), (1, 3, 4), (1, 3, 5), (2, 4, 1), (2, 4, 2), (2, 4, 3), (2, 4, 4), (2, 4, 5)]


[{'a': 1, 'b': 3, 'c': 1},
 {'a': 1, 'b': 3, 'c': 2},
 {'a': 1, 'b': 3, 'c': 3},
 {'a': 1, 'b': 3, 'c': 4},
 {'a': 1, 'b': 3, 'c': 5},
 {'a': 2, 'b': 4, 'c': 1},
 {'a': 2, 'b': 4, 'c': 2},
 {'a': 2, 'b': 4, 'c': 3},
 {'a': 2, 'b': 4, 'c': 4},
 {'a': 2, 'b': 4, 'c': 5}]

In [3]:
nl.format(col1='a', col2='b',val1='v',operator='max')

'What is the max of column a given that b has value v?'

In [5]:
dic0 = {'null':0}
dic1 = {'one':1}
dict(dic0, **dic1)

{'null': 0, 'one': 1}

In [5]:
NOOP.sql is None

False

In [8]:
SQLOperatorTemplate(
                        NOOP.sql,
                        main_expr,
                        brackets=None,
                        over_clause=None
                    ).operator_name

('{col1}',)
('{col1}',)


''

In [9]:
'' or 'sfsf'

'sfsf'

In [10]:
print(None)

None


In [9]:
('a' if False else '') + '' + '{a}' + ('b' if False else '')

'{a}'

In [6]:
[sub_tup for tup in ((1,2),(3,4)) for sub_tup in tup]

[1, 2, 3, 4]

In [8]:
condition = '{sdf'
1 if isinstance(condition, str) and condition.startswith('{') else 0

1

In [20]:
import re
'dasd'.find(re.compile(r'd.*d'))

TypeError: must be str, not re.Pattern

In [8]:
from contexttimer import Timer
loops = 10000
a = 'a'
b = 1
with Timer() as t1:
    for i in range(loops):
        c = [1,2]
        c.append(a)
        c.append(b)

a = ['a',1]
b = [1]
with Timer() as t2:
    c = [1,2]
    for i in range(loops):
        c.extend(a)
        
print(t1.elapsed)
print(t2.elapsed)
        

0.0016081000212579966
0.0009223001543432474


In [14]:
from copy import copy, deepcopy
a = dict(a=[1,2], b=[3,4])
b = list(a.values())

print(a)
b[0].append(5)
print(a)

{'a': [1, 2], 'b': [3, 4]}
{'a': [1, 2, 5], 'b': [3, 4]}


In [4]:
import pickle
from src.data_synthesis import TableQuestionDataSet
tables = list(main().values())
tables_reduced = tables
base_description = \
"""
Basic SQL operators min, max, avg, sum or no operation combined with a simple value lookup condition of a different column.
Using WikiTables test set.

"""
dataset = TableQuestionDataSet('base',
                               description=base_description,
                               question_templates=[basic_template],
                               tables=tables_reduced
                               )
pickle.dump(dataset, open('basic_dataset.pickle','wb'))

 15%|████████████████████▎                                                                                                                 | 63/415 [00:13<00:51,  6.88it/s]/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:1117: UserWarning: Encountered empty value. Consider checkingsampling strategy or table data quality!
  warnings.warn('Encountered empty value. Consider checking'
  0%|▏                                                                                                                                 | 324/171200 [00:04<35:15, 80.77it/s]/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
  0%|▎                                                                                                                                 | 342/171200 [00:04<35:24, 

/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn(

/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
  1%|▊                                                                                                                                | 1145/171200 [00:14<43:50, 64.66it/s]/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
/home/choenes/TabEmbed_workspace/NumTim

/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
  1%|█▏                                                                                                                               | 1554/171200 [00:20<38:10, 74.06it/s]/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
/home/choenes/TabEmbed_workspace/NumTim

/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
  1%|█▋                                                                                                                               | 2288/171200 [00:29<39:11, 71.84it/s]/home/choenes/TabEmbed_workspace/NumTim

...

 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 169318/171200 [39:28<00:24, 76.27it/s]/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
/home/choenes/TabEmbed_workspace/NumTim

 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 169350/171200 [39:28<00:24, 74.09it/s]/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
/home/choenes/TabEmbed_workspace/NumTim

/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 169398/171200 [39:29<00:24, 75.06it/s]/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
 99%|██████████████████████████████████

/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████ | 169892/171200 [39:35<00:16, 79.51it/s]/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
/home/choenes/TabEmbed_workspace/NumTim

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 171153/171200 [39:52<00:00, 81.61it/s]/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 171162/171200 [39:52<00:00, 82.51it/s]/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple rows.Queries that result in a unique answer should be preferred.
  warnings.warn("Query result of dataframe returned multiple rows."
/home/choenes/TabEmbed_workspace/NumTimeProbe/src/data_synthesis.py:339: UserWarning: Query result of dataframe returned multiple 

In [4]:
import pickle
testi = [0]
pickle.dump(testi, open('testi.pickle','wb'))

In [2]:
from src.data_synthesis import main
tables = list(main().values())
for t in tables:
    #if "Toy sales\n(US $ Billions)" in t._data_dict['header'] or "Deaths (age <15)" in t._data_dict['header']:
    #if "Location City or town" in t._data_dict['header'] or 'Date listed' in t._data_dict['header']:
    if "Pole Position" in t._data_dict['header'] and "Report" in t._data_dict['header']:
        print(t._data_dict['header'])
        print(set(t.pandas_dataframe.iloc[:, -1]))
        break
print(t._inferred_column_types)

['Round', 'Round', 'Race', 'Date', 'Pole Position', 'Fastest Lap', 'Winning Club', 'Winning Team', 'Report']
{'Report'}
['numeric', 'text', 'text', 'text', 'text', 'text', 'text', 'text', 'text']


In [3]:
import re
def is_numeric(x):
    regex = re.compile(r'(\d(,\d{3})*|\d+)?(\.\d+)?')
    return re.fullmatch(regex, x) is not None
is_numeric('R2')
t.infer_column_type('Round_2')

'text'

In [2]:
print(type(t._data_dict['header']))
for c, column in enumerate(t._data_dict['header']):
    # empty column names are replaced with column_ + id
    print(column)
    print(column or f'column_{c}')
    t._data_dict['header'][c] = column or f'column_{c}'
t.data_dict['header'][0] = 'efdada'
print(t.data_dict['header'])

b = dict()
a = ['','asds']
b['a'] = a
a[0] = 'a'
b['a']

<class 'list'>

column_0
Name on the Register
Name on the Register
Date listed
Date listed
Location
Location
City or town
City or town
Summary
Summary
['column_0', 'Name on the Register', 'Date listed', 'Location', 'City or town', 'Summary']


['a', 'asds']

In [5]:
t_qs = basic_template.create_questions([t])
#'select... ' + (None or "''")
for tq in t_qs:
    print(tq._sql_query)
    print(tq._nl_question)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.57it/s]

SELECT min("") FROM df
WHERE true
	AND "Location" = 
                                         '1 Market St.\n43°15′54″N 70°51′58″W﻿ / ﻿43.265°N 70.866111°W'
                                         
What is the minimum of column  given that Location has value 
                                         '1 Market St.\n43°15′54″N 70°51′58″W﻿ / ﻿43.265°N 70.866111°W'
                                         ?
SELECT min("") FROM df
WHERE true
	AND "Location" = 
                                         'John Hatch Park\nSouth of Wiswall Road just east of the Lamprey River\n43°06′15″N 70°57′44″W﻿ / ﻿43.1043°N 70.9621°W'
                                         
What is the minimum of column  given that Location has value 
                                         'John Hatch Park\nSouth of Wiswall Road just east of the Lamprey River\n43°06′15″N 70°57′44″W﻿ / ﻿43.1043°N 70.9621°W'
                                         ?
SELECT min("") FROM df
WHERE true
	AND "Location" = 
                   

In [25]:
class SQLColumnExpression:
    pass
def gen():
    return 'qnwkndkjnd'
col_expr = '"Toy sales\n(US $ Billions)"'
val = ''
(f"\n\tAND {col_expr} {'='} " +
    (gen()
     if isinstance(val, SQLColumnExpression)
     else (val or r"''")   # two single quotes as fallback if empty
     )
    )

'\n\tAND "Toy sales\n(US $ Billions)" = \'\''

In [5]:
templi = 'temp_{val}_late'
dosiquo = "''"
templi.format(val=dosiquo)

"temp_''_late"

In [5]:
('12','fdgfegvd') + ('ew', "''")

('12', 'fdgfegvd', 'ew', "''")

In [4]:
dat = (('1','2'),('','4'))
for value_list in dat:
    for value in value_list:
        if value == '':
            value = "''"
dat

(('1', '2'), ('', '4'))

In [5]:
import pickle

from transformers import AutoTokenizer, TapexTokenizer, BartForConditionalGeneration, AutoModelForSeq2SeqLM, T5ForConditionalGeneration
import pandas as pd

from src.data_synthesis import TableQuestionDataSet
from src.evaluation import TableQaModel, SemanticParsingModel, evaluate

#tokenizer = TapexTokenizer.from_pretrained("microsoft/tapex-base-finetuned-wtq")
#model = BartForConditionalGeneration.from_pretrained("microsoft/tapex-base-finetuned-wtq")
#tokenizer = AutoTokenizer.from_pretrained("neulab/omnitab-large-finetuned-wtq")
#model = AutoModelForSeq2SeqLM.from_pretrained("neulab/omnitab-large-finetuned-wtq")
tokenizer = AutoTokenizer.from_pretrained("LarkAI/codet5p-770m_nl2sql_oig")
model = T5ForConditionalGeneration.from_pretrained("LarkAI/codet5p-770m_nl2sql_oig")

#omnitab = TableQaModel(model, tokenizer)
lark_ai = SemanticParsingModel(model, tokenizer)

table_qa_dataset = pickle.load(open('basic_dataset.pickle','rb'))
# due to changes to TableQuestionDataSet saved model is missing property -> set manually
table_qa_dataset._is_answers_computed = True
predict_first = lark_ai.predict([table_qa_dataset._questions[0]._nl_question], [table_qa_dataset._questions[0]._table])
#metric_result, preds, _, _ = evaluate(lark_ai, table_qa_dataset)
#print(metric_result)

In [7]:
#preds
''.join(predict_first)

'SELECT MIN( T2.rank ) FROM csv AS T1 JOIN cyclist AS T2 ON T1.cyclist = T2.cyclist WHERE T1.csvfile = "Davide Rebellin" ;'

In [5]:
[question._nl_question for question in table_qa_dataset.questions[:10]]

["What is the minimum of column Rank given that Cyclist has value \n                                         'Davide Rebellin\xa0(ITA)'\n                                         ?",
 "What is the minimum of column Rank given that Cyclist has value \n                                         'Haimar Zubeldia\xa0(ESP)'\n                                         ?",
 "What is the minimum of column Rank given that Cyclist has value \n                                         'Samuel Sánchez\xa0(ESP)'\n                                         ?",
 "What is the minimum of column Rank given that Cyclist has value \n                                         'Paolo Bettini\xa0(ITA)'\n                                         ?",
 "What is the minimum of column Rank given that Cyclist has value \n                                         'Stéphane Goubert\xa0(FRA)'\n                                         ?",
 "What is the minimum of column Rank given that Cyclist has value \n                        

In [7]:
table_qa_dataset.questions[0]._table.pandas_dataframe

,Rank,Cyclist,Team,Time,UCI ProTour\nPoints
0,1,Alejandro Valverde (ESP),Caisse d'Epargne,"5h 29' 10""",40
1,2,Alexandr Kolobnev (RUS),Team CSC Saxo Bank,s.t.,30
2,3,Davide Rebellin (ITA),Gerolsteiner,s.t.,25
3,4,Paolo Bettini (ITA),Quick Step,s.t.,20
4,5,Franco Pellizotti (ITA),Liquigas,s.t.,15
5,6,Denis Menchov (RUS),Rabobank,s.t.,11
6,7,Samuel Sánchez (ESP),Euskaltel-Euskadi,s.t.,7
7,8,Stéphane Goubert (FRA),Ag2r-La Mondiale,"+ 2""",5
8,9,Haimar Zubeldia (ESP),Euskaltel-Euskadi,"+ 2""",3
9,10,David Moncoutié (FRA),Cofidis,"+ 2""",1


In [29]:
import torch
from transformers import T5ForConditionalGeneration, AutoTokenizer

device = torch.device("cpu")

tokenizer = AutoTokenizer.from_pretrained("LarkAI/codet5p-770m_nl2sql_oig")
model = T5ForConditionalGeneration.from_pretrained("LarkAI/codet5p-770m_nl2sql_oig").to(device)

text = "Given the following schema:\ntrack (Track_ID, Name, Location, Seating, Year_Opened)\nrace (Race_ID, Name, Class, Date, Track_ID)\nWrite a SQL query to count the number of tracks."
text = [text, 'davsdcsa bdjsa dasndkas']
#inputs = tokenizer.encode(text, padding=True, truncation=True, return_tensors="pt").to(device)
inputs2 = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(device)
print(type(inputs2['input_ids']))
output_ids = model.generate(inputs2['input_ids'], max_length=512)
print(type(output_ids))
response_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
# SELECT COUNT( * ) FROM track

<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [30]:
response_text

['SELECT COUNT( * ) FROM track ;',
 'SELECT t1.name FROM ducks AS t1 JOIN bdjsa AS t2 ON t1.bdjsa = t2.bdjsa WHERE t2.name = "davidcsa" ;']

In [28]:
#print('inputs:\n', inputs)
print('inputs2:\n', inputs2)
print('output_ids:\n', output_ids)

inputs2:
 {'input_ids': tensor([[    1,  6083,   326,  3751,  1963,    30,   203,  4101,   261,  4402,
            67,   734,    16,  1770,    16,  7050,    16,  3265,  1776,    16,
         16666,    67, 23115,    13,   203,  9963,   261,    54,   623,    67,
           734,    16,  1770,    16,  1659,    16,  2167,    16, 11065,    67,
           734,    13,   203,  3067,   279,  3063,   843,   358,  1056,   326,
          1300,   434, 13933,    18,     2],
        [    1, 20752,  6427,  2143,    69, 13837,  2924,    69, 30255,    82,
          2883,   345,     2,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1